In [51]:
!pip install -q kaggle

In [52]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True) 

Mounted at /content/gdrive/


In [53]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [54]:
from sklearn import preprocessing
from tensorflow.python import keras
from sklearn.impute import SimpleImputer
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout

In [55]:
!cp "/content/gdrive/My Drive/Colab Notebooks/Projects/kaggle.json" .

In [56]:
!mkdir -p ~/.kaggle

In [57]:
!cp kaggle.json ~/.kaggle/

In [58]:
!chmod 600 ~/.kaggle/kaggle.json

In [59]:
!kaggle competitions download -c titanic

train.csv: Skipping, found more recently modified local copy (use --force to force download)
test.csv: Skipping, found more recently modified local copy (use --force to force download)
gender_submission.csv: Skipping, found more recently modified local copy (use --force to force download)


In [60]:
# Import train & test data 
test = pd.read_csv('/content/test.csv')
train = pd.read_csv('/content/train.csv')

In [61]:
# View the training data
train.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [62]:
test.head(5)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [63]:
encoder = preprocessing.LabelEncoder()

In [64]:
cat_features = ["Sex", "Embarked"]

In [65]:
train["Embarked"] = train["Embarked"].astype(str)

In [66]:
encoded_train = train[cat_features].apply(encoder.fit_transform)
encoded_test = test[cat_features].apply(encoder.fit_transform)

In [67]:
num_features = ["Survived","Pclass","Age","SibSp","Parch","Fare"]
test_features= ["Pclass","Age","SibSp","Parch","Fare"]

In [68]:
training_data = train[num_features].join(encoded_train)
test_data = test[test_features].join(encoded_test)

In [69]:
training_data.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex,Embarked
0,0,3,22.0,1,0,7.2500,1,2
1,1,1,38.0,1,0,71.2833,0,0
2,1,3,26.0,0,0,7.9250,0,2
3,1,1,35.0,1,0,53.1000,0,2
4,0,3,35.0,0,0,8.0500,1,2


In [70]:
test_data.head()

,Pclass,Age,SibSp,Parch,Fare,Sex,Embarked
0,3,34.5,0,0,7.8292,1,1
1,3,47.0,1,0,7.0000,0,2
2,2,62.0,0,0,9.6875,1,1
3,3,27.0,0,0,8.6625,1,2
4,3,22.0,1,1,12.2875,0,2


In [71]:
training_data.isnull().sum()

Survived      0
Pclass        0
Age         177
SibSp         0
Parch         0
Fare          0
Sex           0
Embarked      0
dtype: int64

In [72]:
test_data.isnull().sum()

Pclass       0
Age         86
SibSp        0
Parch        0
Fare         1
Sex          0
Embarked     0
dtype: int64

In [73]:
my_imputer = SimpleImputer()
imputed_train = pd.DataFrame(my_imputer.fit_transform(training_data))
imputed_test_data = pd.DataFrame(my_imputer.fit_transform(test_data))

In [74]:
imputed_train.head()

,0,1,2,3,4,5,6,7
0,0.0,3.0,22.0,1.0,0.0,7.2500,1.0,2.0
1,1.0,1.0,38.0,1.0,0.0,71.2833,0.0,0.0
2,1.0,3.0,26.0,0.0,0.0,7.9250,0.0,2.0
3,1.0,1.0,35.0,1.0,0.0,53.1000,0.0,2.0
4,0.0,3.0,35.0,0.0,0.0,8.0500,1.0,2.0


In [75]:
imputed_train.columns = training_data.columns
imputed_test_data.columns = test_data.columns

In [76]:
imputed_train.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex,Embarked
0,0.0,3.0,22.0,1.0,0.0,7.2500,1.0,2.0
1,1.0,1.0,38.0,1.0,0.0,71.2833,0.0,0.0
2,1.0,3.0,26.0,0.0,0.0,7.9250,0.0,2.0
3,1.0,1.0,35.0,1.0,0.0,53.1000,0.0,2.0
4,0.0,3.0,35.0,0.0,0.0,8.0500,1.0,2.0


In [77]:
X = imputed_train.drop("Survived", axis = 1)
y = imputed_train["Survived"]

In [78]:
tf.random.set_seed(42)

In [79]:
model = Sequential()

In [80]:
model.add(Dense(7, activation = "relu", input_shape = (7,)))
model.add(Dropout(0.2))

In [81]:
model.add(Dense(50, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(50, activation = "relu"))

In [82]:
model.add(Dense(2, activation = "softmax"))

In [83]:
model.compile(loss=keras.losses.sparse_categorical_crossentropy, optimizer = "adam", metrics = ['accuracy'])

In [84]:
model.fit(X, y,
          batch_size=1,
          epochs=700,
          validation_split = 0.2,
          verbose = 0)

In [86]:
preds = model.predict(imputed_test_data)
print(preds)

[[7.43901551e-01 2.56098479e-01]
 [3.42808813e-01 6.57191217e-01]
 [6.95310831e-01 3.04689199e-01]
 [7.45227814e-01 2.54772156e-01]
 [1.89237937e-01 8.10762048e-01]
 [7.32963026e-01 2.67037004e-01]
 [8.56571421e-02 9.14342940e-01]
 [6.82197452e-01 3.17802578e-01]
 [7.96694309e-02 9.20330584e-01]
 [7.56297410e-01 2.43702605e-01]
 [7.48401225e-01 2.51598716e-01]
 [6.05851710e-01 3.94148260e-01]
 [7.96694309e-02 9.20330584e-01]
 [7.12539792e-01 2.87460208e-01]
 [7.96694309e-02 9.20330584e-01]
 [7.96694309e-02 9.20330584e-01]
 [6.66334808e-01 3.33665222e-01]
 [7.22611308e-01 2.77388692e-01]
 [1.70463488e-01 8.29536498e-01]
 [1.06451645e-01 8.93548369e-01]
 [5.67242742e-01 4.32757288e-01]
 [4.64437753e-02 9.53556240e-01]
 [7.96694309e-02 9.20330584e-01]
 [3.50063980e-01 6.49935961e-01]
 [7.96694309e-02 9.20330584e-01]
 [7.74009347e-01 2.25990653e-01]
 [7.96694309e-02 9.20330584e-01]
 [7.24064171e-01 2.75935799e-01]
 [5.70610285e-01 4.29389745e-01]
 [7.48722196e-01 2.51277775e-01]
 [6.994820

In [87]:
predictions = np.array(model.predict(imputed_test_data)).argmax(axis=1)

In [88]:
print(predictions)

[0 1 0 0 1 0 1 0 1 0 0 0 1 0 1 1 0 0 1 1 0 1 1 1 1 0 1 0 0 0 0 0 1 1 1 0 1
 1 0 0 0 0 0 1 1 0 1 0 1 1 1 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 1 1 1 0 1 1
 1 1 0 1 0 1 1 0 0 0 0 0 1 1 1 1 1 0 1 0 1 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 1 1 1 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1 0 1 1 1 0 0 0 1 0
 0 0 1 0 0 1 0 0 1 1 0 1 1 1 1 0 0 1 1 0 1 1 0 0 0 0 0 1 1 0 1 1 0 1 1 0 1
 0 1 0 0 0 0 0 0 0 1 0 1 1 0 1 1 1 1 1 0 1 1 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0
 1 0 1 1 0 1 0 0 0 1 0 0 1 0 0 0 1 1 1 1 1 0 0 0 1 0 1 1 1 0 1 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0 1 1 1 1 1 0 0 1 0 0 1 1 0 0 0 0
 1 0 1 0 0 0 0 0 1 1 1 1 0 1 0 0 0 1 1 1 1 0 0 0 0 0 0 0 1 0 1 1 0 0 1 1 0
 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 0 0 1 1 0
 0 1 0 0 1 1 1 0 0 0 0 0 1 1 0 1 0 0 0 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 0 0
 1 1 1 1 1 1 0 1 0 0 0]
